In [ ]:
#------------------------------------------------------------------------------------------
# 1.임베딩 및 검색 환경 설정 
# => 임베딩 모델 로딩.
# => 엘라스틱서치:ES 연결
# => 각 인스턴스와 변수들 설정
#------------------------------------------------------------------------------------------
import os
import time
import numpy as np
import sys
import json
from tqdm import tqdm 
import numpy as np
import pandas as pd

from embedding import embedding_pdf
from utils import MyUtils
from utils import weighted_reciprocal_rank_fusion, clustering_embedding

from es_8 import My_ElasticSearch 
from langchain_community.embeddings import HuggingFaceEmbeddings, HuggingFaceBgeEmbeddings

from vision import MY_Vision
#-----------------------------------------------------------------------

# 환경설정 파일 로딩
myutils = MyUtils(yam_file_path='../data/docs_settings.yaml')
settings = myutils.get_options()

#-----------------------------------------------------------------------

# 임베딩 모델 로딩
embeding_model = "bongsoo/kpf-sbert-128d-v1"
embedding = HuggingFaceEmbeddings(
    model_name=settings["EMBEDDING_MODEL"], 
    show_progress=False
)

# 샘플 텍스트 데이터를 입력하여 모델이 출력 shape(128인지, 768인지) 확인함
sample_text = ["모델의 출력 shape를 확인합니다."]
# 텍스트 데이터를 임베딩 모델에 입력
sample_embedding_vector = embedding.embed_documents(sample_text)
model_shape = len(sample_embedding_vector[0])
print(f'*임베딩 모델: {embedding.model_name}/*shape:{model_shape}')
#-----------------------------------------------------------------------

# elasticsearch 서버 연결
esindex="mpower10u_image"
es_url=settings['ES_URL']
es_index_file_path=settings['ES_INDEX_FILE_PATH']
es_api_key=settings['ES_API_KEY']  # *elasticsearch 8.x 버전에서는 ES_API_KEY를 발급받아서 연결해야 함.

# es 8.x 버전이면 es_8 패키지 로딩(*단 로딩을 위해서는 # pip로 elasticsearch 8.14.3으로 업데이트 =>!pip install --upgrade elasticsearch)
myes = My_ElasticSearch(es_url=es_url, index_file_path=es_index_file_path, api_key=es_api_key)
print(f'*엘라스틱서치: {myes}')

#-----------------------------------------------------------------------

# vision 모델 로딩
myvision = MY_Vision(model_folder_path=settings['VISION_MODEL'], device=settings['VISION_DEVICE'])
#-----------------------------------------------------------------------

#-----------------------------------------------
# Mpower Synap 추가
from os import sys
sys.path.append('../../../MpowerAI')
from pympower.classes.mshaai import MShaAI

shaai = MShaAI()
#-----------------------------------------------

# global 인스턴스 dict로 정의
global_instance:dict = {'myutils': myutils, 'settings': settings, "embedding": embedding, "myes": myes, "model_shape": model_shape, 'myvision': myvision, "shaai": shaai}

# 출력
search_type:list = ['*벡터검색', '*RRF검색(벡터+BM25)', '*BM25검색']
print(f'*검색수(K)={settings["SEARCH_K"]}')
print(f'*검색방식={settings["RRF_SEARCH"]}/{search_type[settings["RRF_SEARCH"]]}')


In [ ]:
#------------------------------------------------------------------------------------------
# 문서 임베딩
# => text 추출된 문서가 있는 폴더를 지정하면 인덱싱 함.
# => 하나의 문서에 평균벡터(vector0) 인덱싱됨.
#------------------------------------------------------------------------------------------
from docs_func import embedding_file_list_doc

response:dict = {}

indexing=True    # True=elasticsearch로 인덱싱. False=vector값만 리턴
del_index=True   # True=기존 elasticsearch에 같은 index명이 있으면 제거. False=제거하지 않고 추가.     
file_folder = "../../sample/extra/"  # text 추출된 인덱싱할 폴더

file_path_list = myutils.getListOfFiles(file_folder) # 폴더에 파일 path list를 얻어옴.

response = embedding_file_list_doc(instance=global_instance, 
                                index_name=esindex,   # 인덱스명
                                file_path_list=file_path_list, # 임덱싱할 파일들 경로.
                                indexing=indexing,    # True=elasticsearch로 인덱싱. False=vector값만 리턴
                                del_index=del_index)  # True=기존 elasticsearch에 같은 index명이 있으면 제거. False=제거하지 않고 추가.     

print(response)

In [ ]:
#------------------------------------------------------------------------------------------
# 검색 테스트
# =>search_rfile_name_list = 검색할 문서 rfile_name들(*없으면 모든 rfile_name에 대해 검색)
# =>rfile_type = 0,1만 입력=>0=신규문서(query_rfile=문서내용 입력됨) 1=기존문서(query_rfile=rfile_name 명 입력됨)
# =>query_rfile = 신규 문서면 문서내용(*단 이미지면 이미지경로가 들어감)  혹은 기존 문서면 rfile_name
#------------------------------------------------------------------------------------------
from docs_func import search_docs, mpower_save_docs, extract_save_doc01, is_image_file
from utils import generate_random_string 

response:dict = {}
file_path:str = ""
    
# == 인자들 설정 ===================
save_folder_path = "./dm"        # 신규문서일때 문서가 저장될 경로
search_rfile_name_list:list = []        # 검색할 문서 rfile_name들(*없으면 모든 rfile_name에 대해 검색)
rfile_type:int = 0                      # 0,1만 입력=>0=신규문서(query_rfile_text=문서내용 입력됨) 1=기존문서(query_rfile_text=rfile_name 명 입력됨)
query_rfile:str = "../../sample/eval/동물+사람.jpg"  # 신규 문서 0이면 문서내용(*단 이미지면 이미지경로가 들어감) 혹은 기존 문서면 rfile_name

search_k = 3       # 검색수
search_method = 0  # 검색방식(0=vector 검색, 1=vector+bm25 검색, 2=bm25검색)

# == 신규문서 or 기존문서 처리==============
# rfile_type==0, 즉 신규문서내용이 입력된 경우에만 파일명으로 저장.
if rfile_type == 0:
    random_file_name = generate_random_string(10) # 10자리 랜덤한 파일명 만듬.
        
    # 임미지이면 이미지 추출후 파일 저장
    if is_image_file(query_rfile)==True:
        # tgt 폴더가 없으며 새로 생성.
        if not os.path.exists(save_folder_path):
            os.makedirs(save_folder_path)
            
        # tgt 파일 지정 하고 이미지 추출후 저장    
        tgtFilePath = save_folder_path + "/" + random_file_name 
        print(f'*extract_save_doc01=>tgtFilePath:{tgtFilePath}, srcPath:{query_rfile}')
        
        error, imagetotext=extract_save_doc01(instance=global_instance, srcPath=query_rfile, tgtPath=tgtFilePath, mime_type="img")
        
        print(f'*extract_save_doc01=>error:{error}, text:{imagetotext}')
        
        # 추출된 파일 경로 지정
        file_path=tgtFilePath
    else:
        # query_rfile_text(문서쿼리내용) 랜덤한 파일명으로 저장
        print(f'*mpower_save_docs=>random_file_name:{random_file_name}')
        res=mpower_save_docs(folder_path=save_folder_path, 
                                rfile_name_list=[random_file_name], 
                                rfile_text_list=[query_rfile])
    
        file_path = res['file_path_list'][0]
        
# rfile_type==1, 즉 기존문서 rfile_name이 입력된 경우에는 file_path를 rfile_name으로 지정
elif rfile_type == 1:
    file_path = query_rfile
        
# == 임베딩 & 검색==============
response = search_docs(instance=global_instance, 
                           index_name=esindex, 
                           file_type=rfile_type,
                           file_path=file_path, 
                           uids=search_rfile_name_list, 
                           search_k=search_k, 
                           search_method=search_method)

print(response)